## We make a file for the galaxy number density distributions for different tracers. 
## Take reference from Validation of the Scientific Program for the Dark Energy Spectroscopic Instrument (arXiv:2306.06307)--08-19-2023
Take reference from /home/zjding/csst_bao/fisher_pkmu/parameter_preparation/Sigma_nl_linear_order.ipynb

Derive the relation between the angular number density $\frac{dN}{dzd\Omega}$ and the volume number density $dN/dV$:
\begin{align}
f_1 = \frac{dN}{dzd\Omega}\\
f_2 = dN/dV = \frac{dN}{s^2d\Omega ds}=f_1 \frac{dz}{s^2ds}.
\end{align}
We have 
\begin{align}
s = \int\frac{cdz}{H_0\sqrt{(1+z)^3 \Omega_m + \Omega_{\Lambda}}},
\end{align}
hence,
\begin{align}
f_2 = \frac{f_1}{c s^2} H_0\sqrt{(1+z)^3 \Omega_m + \Omega_{\Lambda}}.
\end{align}

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import sys
sys.path.append("/home/zjding/DESI_mockchallenge/bao_fit/pymodule_barn")
from mcmc_funs import growth_factor
from scipy import interpolate, integrate

In [3]:
def Sigma2_dd_integrand(k, splPlin, R_bao, Sigma2_sm):
    Pk_lin_0 = splPlin(k)
    Sm_kernel = np.exp(-0.5*k*k * Sigma2_sm)        # 1/2 factor consistent with the convention of the smoothing factor in MC
    return Pk_lin_0*(1.0-Sm_kernel)**2.0

def cal_growth_rate(a_z, omega_m):
    a_z_p = a_z+0.01
    a_z_m = a_z-0.01
    D_p = growth_factor(1./a_z_p-1, omega_m)
    D_m = growth_factor(1./a_z_m-1, omega_m)
    f = (np.log(D_p)-np.log(D_m))/(np.log(a_z_p) - np.log(a_z_m))
    return f

def comoving_dis_fun(z, Omega_m, speed_c):
    res = speed_c/(100.0 * (Omega_m*(1+z)**3 + (1-Omega_m))**0.5) 
    return res

def volden_from_surfaceden(f1, z, Omega_m, speed_c):
    s = integrate.quad(comoving_dis_fun, 0., z, args=(Omega_m, speed_c))[0]
    f2 = f1 / (s**2.0 * comoving_dis_fun(z, Omega_m, speed_c))
    return f2

In [4]:
Omega_m = 0.3075
speed_c = 299792.458    # speed of light, km/s
skyarea_total = 4*np.pi * (180./np.pi)**2.0    # unit: deg^2
print(skyarea_total)
deg2_per_str = 1/(180./np.pi)**2.0    # deg^2 to steradian 

41252.96124941928


In [5]:
# input (theoretical) linear power spectrum
ifile_pklin = "/home/zjding/DESI_mockchallenge/bao_fit/stage2_3Gpc_postrec/data/plin_model/Pk_Planck15_Table4.txt"
kwig, Plin_wig = np.loadtxt(ifile_pklin, dtype='f8', comments='#', unpack=True) 
splPlin = interpolate.InterpolatedUnivariateSpline(kwig, Plin_wig)

## using Table 7 of the paper
using Table 2.3 of DESI white paper (arXiv:1611.00036) gives under-estimation on LRG number density

In [6]:
def param_workshop(z_input, dN_dOmega, dz, b_0, z_output):
    #z_input = (zmin_array+zmax_array)/2.0
    G_0 = growth_factor(0.0, Omega_m)                 # G_0 at z=0, normalization factor
    Dz_array = growth_factor(z_output, Omega_m)/G_0   # the normalized linear growth factor
    bias_array = b_0/Dz_array
    
    a_array = 1./(1+z_output)
    f_array = cal_growth_rate(a_array, Omega_m)
    
    dN_dOmegadz = dN_dOmega/dz
    dN_dstrdz = dN_dOmegadz /deg2_per_str
    spl_f1 = interpolate.InterpolatedUnivariateSpline(z_input, dN_dstrdz)
    
    # output n(z) galaxy volume number density
    nz_list = []
    for z in z_output:
        nz = volden_from_surfaceden(spl_f1(z), z, Omega_m, speed_c)
        nz_list.append(nz)
    n_z = np.array(nz_list)
    
    ## calculate the nonlinear BAO damping scales Sigma_xy, Sigma_z
    q_max = 100.0
    Sigma_sm = 1.e4    # a very large number for pre-recon
    Sigma_z_list = []
    Sigma_xy_list = []
    for z, f in zip(z_output, f_array):
        print("z, f: %.2f, %.3f"%(z, f))
        norm_gf = growth_factor(z, Omega_m)/G_0
        const = 1.0/(6.0*np.pi**2.0) * norm_gf**2.0

        Sigma2_sm = Sigma_sm **2.0
        Sigma2_dd = const * integrate.quad(Sigma2_dd_integrand, kwig[0], 100.0, args=(splPlin, q_max, Sigma2_sm), epsabs=1.e-4, epsrel=1.e-4)[0]
        Sigma_xy = (2.0*Sigma2_dd)**0.5         # There is a factor of 2 due to different expression between Eq. 84 of Blanchard paper and the usual one in the BOSS analysis.
        Sigma_z = (1.0+f)*Sigma_xy
        print("Sigma_z, Sigma_xy:", Sigma_z, Sigma_xy)
        Sigma_z_list.append(Sigma_z)
        Sigma_xy_list.append(Sigma_xy)
    
    
    return n_z, bias_array, f_array, np.array(Sigma_z_list), np.array(Sigma_xy_list)

In [7]:
tracer_list = ['BGS', 'LRG', 'ELG_LOP', 'QSO']
zmin_in_dict = {'BGS': np.arange(0., 0.31, 0.1), 'LRG': np.arange(0.4, 1.01, 0.1),
             'ELG_LOP': np.arange(1.1, 1.51, 0.1), 'QSO': np.arange(1.6, 2.01, 0.1)}
zmax_in_dict = {'BGS': np.arange(0.1, 0.41, 0.1), 'LRG': np.arange(0.5, 1.11, 0.1),
             'ELG_LOP': np.arange(1.2, 1.61, 0.1), 'QSO': np.arange(1.7, 2.11, 0.1)}
dN_dOmega_dict = {'BGS': np.array([101.1, 231.3, 216.9, 97.3]),
                  'LRG': np.array([47.5, 65.6, 80.0, 93.2, 99.3, 63.7, 28.3]),
                  'ELG_LOP': np.array([108.0, 103.6, 97.1, 87.7, 55.4]),
                  'QSO': np.array([12.1, 11.8, 11.1, 10.6, 9.5])}
dz = 0.1
b0_dict = {'BGS': 1.34, 'LRG': 1.7, 'ELG_LOP': 0.84, 'QSO': 1.2}


## set output zbins to match with part of redshift bins of CSST, Euclid, Roman 
zmin_out_dict = {'BGS': np.array([0.0, 0.2]), 'LRG': np.array([0.4, 0.6, 0.8]),
                 'ELG_LOP': np.array([1.1, 1.3]), 'QSO': np.array([1.6, 1.8])}
zmax_out_dict = {'BGS': np.array([0.2, 0.4]), 'LRG': np.array([0.6, 0.8, 1.1]),
                 'ELG_LOP': np.array([1.3, 1.6]), 'QSO': np.array([1.8, 2.1])}

In [8]:
header = "z_low   z_up   n(z) [h/Mpc]^3   bias    growth_rate(f)     Sigma_z     Sigma_xy"
output = np.empty((0, 7))
for tracer in tracer_list:
    zmin_in = zmin_in_dict[tracer]
    zmax_in = zmax_in_dict[tracer]
    zmid_in = (zmin_in + zmax_in)/2.0
    dN_dOmega = dN_dOmega_dict[tracer]
    b0 = b0_dict[tracer]
    
    zmid_out = zmid_in
    nz_, bias_, f_, Sigmaz_, Sigmaxy_ = param_workshop(zmid_in, dN_dOmega, dz, b0, zmid_out)
    temp = np.array([zmin_in, zmax_in, nz_, bias_, f_, Sigmaz_, Sigmaxy_]).T 
    
    ofile = f"./input/nz_bias_f_Sigmanl_DESI_{tracer}_Adame_2023a.txt"
    np.savetxt(ofile, temp, fmt="%.7e", header=header, comments='#')
    
    output = np.vstack((output, temp))

z, f: 0.05, 0.550
Sigma_z, Sigma_xy: 12.61220647698538 8.137685127695319
z, f: 0.15, 0.605
Sigma_z, Sigma_xy: 12.395729236948172 7.721089043102608
z, f: 0.25, 0.655
Sigma_z, Sigma_xy: 12.126057327511521 7.325684554314015
z, f: 0.35, 0.699
Sigma_z, Sigma_xy: 11.816433678882081 6.953520345444755
z, f: 0.45, 0.738
Sigma_z, Sigma_xy: 11.479422995893685 6.605374307219208
z, f: 0.55, 0.771
Sigma_z, Sigma_xy: 11.126026763225271 6.281097439291702
z, f: 0.65, 0.800
Sigma_z, Sigma_xy: 10.765303451549093 5.979909250918312
z, f: 0.75, 0.825
Sigma_z, Sigma_xy: 10.404341996434216 5.700633096006943
z, f: 0.85, 0.847
Sigma_z, Sigma_xy: 10.048441331882573 5.441872665758699
z, f: 0.95, 0.865
Sigma_z, Sigma_xy: 9.701383309560105 5.202137597848794
z, f: 1.05, 0.881
Sigma_z, Sigma_xy: 9.365725864069223 4.979928488756738
z, f: 1.15, 0.894
Sigma_z, Sigma_xy: 9.043075815983187 4.773791332828013
z, f: 1.25, 0.906
Sigma_z, Sigma_xy: 8.734323134761887 4.582349947767653
z, f: 1.35, 0.916
Sigma_z, Sigma_xy: 8.4398

In [9]:
ofile = "./input/nz_bias_f_Sigmanl_DESI_alltracers_Adame_2023a.txt"
np.savetxt(ofile, output, fmt="%.7e", header=header, comments='#')

## for the output with rebinning

In [10]:
output = np.empty((0, 7))
odir = "./input/rebin/"
for tracer in tracer_list:
    zmin_in = zmin_in_dict[tracer]
    zmax_in = zmax_in_dict[tracer]
    zmid_in = (zmin_in + zmax_in)/2.0
    dN_dOmega = dN_dOmega_dict[tracer]
    b0 = b0_dict[tracer]
    
    zmin_out = zmin_out_dict[tracer]
    zmax_out = zmax_out_dict[tracer]
    zmid_out = (zmin_out + zmax_out)/2.0
    
    nz_, bias_, f_, Sigmaz_, Sigmaxy_ = param_workshop(zmid_in, dN_dOmega, dz, b0, zmid_out)
    temp = np.array([zmin_out, zmax_out, nz_, bias_, f_, Sigmaz_, Sigmaxy_]).T 
    
    ofile = odir + f"nz_bias_f_Sigmanl_DESI_{tracer}_Adame_2023a.txt"
    np.savetxt(ofile, temp, fmt="%.7e", header=header, comments='#')
    
    output = np.vstack((output, temp))
ofile = odir + "nz_bias_f_Sigmanl_DESI_alltracers_Adame_2023a.txt"
np.savetxt(ofile, output, fmt="%.7e", header=header, comments='#')   

z, f: 0.10, 0.578
Sigma_z, Sigma_xy: 12.511434950564029 7.9269257926640915
z, f: 0.30, 0.678
Sigma_z, Sigma_xy: 11.975425109040154 7.13662322802798
z, f: 0.50, 0.755
Sigma_z, Sigma_xy: 11.30415973175205 6.4402892653369195
z, f: 0.70, 0.813
Sigma_z, Sigma_xy: 10.584482316248536 5.8376159862605235
z, f: 0.95, 0.865
Sigma_z, Sigma_xy: 9.701383309560258 5.202137597848797
z, f: 1.20, 0.900
Sigma_z, Sigma_xy: 8.886929168555357 4.676316090958426
z, f: 1.45, 0.925
Sigma_z, Sigma_xy: 8.159591267167423 4.2385316877117685
z, f: 1.70, 0.943
Sigma_z, Sigma_xy: 7.519143190800064 3.8707574590508727
z, f: 1.95, 0.955
Sigma_z, Sigma_xy: 6.957794472059039 3.558740626464466
